In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.applications import ResNet50
from keras.layers import GlobalAveragePooling2D, Dense, BatchNormalization, Dropout
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report

In [2]:
from google.colab import drive
drive.mount('/content/drive')

train_dir = "/content/drive/MyDrive/Projeto_Pneumonia/chest_xray/train"
test_dir = "/content/drive/MyDrive/Projeto_Pneumonia/chest_xray/test"
val_dir = "/content/drive/MyDrive/Projeto_Pneumonia/chest_xray/val"

Mounted at /content/drive


In [3]:
#Gerador de imagens pré-processamento
image_generator = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    samplewise_center=True,
    samplewise_std_normalization=True
)

In [4]:
#Gerador dados treinamento e validação
train_generator = image_generator.flow_from_directory(
    train_dir,
    batch_size=8,
    shuffle=True,
    class_mode='binary',
    target_size=(180, 180)
)

validation_generator = image_generator.flow_from_directory(
    val_dir,
    batch_size=1,
    shuffle=False,
    class_mode='binary',
    target_size=(180, 180)
)


Found 5218 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


In [5]:
#Resnet
resnet_base_model = ResNet50(
    input_shape=(180, 180, 3),
    include_top=False,
    weights='imagenet'
)

94765736/94765736 [==============================] - 4s 0us/step


In [6]:
#Camadas personalizadas
resnet_model = tf.keras.Sequential([
    resnet_base_model,
    GlobalAveragePooling2D(),
    Dense(512, activation="relu"),
    BatchNormalization(),
    Dropout(0.6),
    Dense(128, activation="relu"),
    BatchNormalization(),
    Dropout(0.4),
    Dense(64, activation="relu"),
    BatchNormalization(),
    Dropout(0.3),
    Dense(1, activation="sigmoid")
])

In [7]:
#Compilando Resnet
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
METRICS = [
    'accuracy',
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall'),
]
resnet_model.compile(optimizer=opt, loss='binary_crossentropy', metrics=METRICS)

In [8]:
# Treinando modelo

# Calcular o peso das classes com base na contagem
num_normal = len(os.listdir(os.path.join(train_dir, 'NORMAL')))
num_pneumonia = len(os.listdir(os.path.join(train_dir, 'PNEUMONIA')))

weight_for_0 = num_pneumonia / (num_normal + num_pneumonia)
weight_for_1 = num_normal / (num_normal + num_pneumonia)

class_weight = {0: weight_for_0, 1: weight_for_1}

history = resnet_model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    class_weight=class_weight,
    steps_per_epoch=100,
    validation_steps=25
)

Epoch 1/10
100/100 [==============================] - ETA: 0s - loss: 0.3000 - accuracy: 0.5700 - precision: 0.8093 - recall: 0.5705

100/100 [==============================] - 594s 6s/step - loss: 0.3000 - accuracy: 0.5700 - precision: 0.8093 - recall: 0.5705 - val_loss: 1.2056 - val_accuracy: 0.5000 - val_precision: 0.5000 - val_recall: 1.0000
Epoch 2/10
100/100 [==============================] - 491s 5s/step - loss: 0.2499 - accuracy: 0.6725 - precision: 0.8638 - recall: 0.6582
Epoch 3/10
100/100 [==============================] - 484s 5s/step - loss: 0.2220 - accuracy: 0.6888 - precision: 0.9005 - recall: 0.6600
Epoch 4/10
100/100 [==============================] - 487s 5s/step - loss: 0.2108 - accuracy: 0.7163 - precision: 0.8904 - recall: 0.6964
Epoch 5/10
100/100 [==============================] - 489s 5s/step - loss: 0.1865 - accuracy: 0.7513 - precision: 0.9304 - recall: 0.7265
Epoch 6/10
100/100 [==============================] - 493s 5s/step - loss: 0.2101 - accuracy: 0.7075 - precision: 0.8973 - recall: 0.6982
Epoch 7/10
100/100 [==============================] - 495s 5s/step - loss: 0.2071 - accuracy: 0.

In [9]:
# Métricas para treinamento
train_metrics = resnet_model.evaluate(train_generator, steps=len(train_generator))
train_loss, train_accuracy, train_precision, train_recall = train_metrics

# Métricas para validação
val_metrics = resnet_model.evaluate(validation_generator, steps=len(validation_generator))
val_loss, val_accuracy, val_precision, val_recall = val_metrics

# Calcular F1 Score manualmente para treinamento
f1_train = 2 * (train_precision * train_recall) / (train_precision + train_recall)

# Calcular F1 Score manualmente para validação
f1_val = 2 * (val_precision * val_recall) / (val_precision + val_recall)

# Exibir métricas
print("Métricas de Treinamento:")
print(f"Loss: {train_loss:.4f}")
print(f"Acurácia: {train_accuracy * 100:.2f}%")
print(f"Precisão: {train_precision * 100:.2f}%")
print(f"Recall: {train_recall * 100:.2f}%")
print(f"F1 Score (Treinamento): {f1_train * 100:.2f}%")

print("\nMétricas de Validação:")
print(f"Loss: {val_loss:.4f}")
print(f"Acurácia: {val_accuracy * 100:.2f}%")
print(f"Precisão: {val_precision * 100:.2f}%")
print(f"Recall: {val_recall * 100:.2f}%")
print(f"F1 Score (Validação): {f1_val * 100:.2f}%")

16/16 [==============================] - 3s 155ms/step - loss: 0.8979 - accuracy: 0.6250 - precision: 0.5714 - recall: 1.0000
Métricas de Treinamento:
Loss: 0.5980
Acurácia: 79.28%
Precisão: 97.01%
Recall: 74.41%
F1 Score (Treinamento): 84.22%

Métricas de Validação:
Loss: 0.8979
Acurácia: 62.50%
Precisão: 57.14%
Recall: 100.00%
F1 Score (Validação): 72.73%
